In [12]:
# try everything here
import sys
sys.path.append('../')

# other
import pandas as pd
import numpy as np

# utils
from src.data.make_dataset import load_data#, merge_all_datasets



In [13]:
from src.data.make_dataset import *

In [15]:
merge_all_datasets()

NameError: name 'merge_all_datasets' is not defined

In [3]:
# Loading Data
data_dict = load_data()

olist_customers_df = data_dict['olist_customers_df']
olist_geolocation_df = data_dict['olist_geolocation_df']
olist_order_items_df = data_dict['olist_order_items_df']
olist_order_payments_df = data_dict['olist_order_payments_df']
olist_order_reviews_df = data_dict['olist_order_reviews_df']
olist_orders_df = data_dict['olist_orders_df']
olist_products_df = data_dict['olist_products_df']
olist_sellers_df = data_dict['olist_sellers_df']
product_category_name_translation_df = data_dict['product_category_name_translation_df']

olist_closed_deals_df = data_dict['olist_closed_deals_df']
olist_marketing_qualified_leads_df = data_dict['olist_marketing_qualified_leads_df']

In [4]:
# Merging into one

def merge_all_datasets(olist_customers_df: pd.DataFrame, 
                       olist_geolocation_df: pd.DataFrame,
                       olist_order_items_df: pd.DataFrame,
                       olist_order_payments_df: pd.DataFrame,
                       olist_order_reviews_df: pd.DataFrame, 
                       olist_orders_df: pd.DataFrame,
                       olist_products_df: pd.DataFrame, 
                       olist_sellers_df: pd.DataFrame, 
                       olist_closed_deals_df: pd.DataFrame, 
                       olist_marketing_qualified_leads_df: pd.DataFrame) -> pd.DataFrame:

    df = olist_orders_df.merge(olist_order_items_df, on='order_id', how='left')
    df = df.merge(olist_order_payments_df, on='order_id', how='outer', validate='m:m')
    df = df.merge(olist_order_reviews_df, on='order_id', how='outer')
    df = df.merge(olist_products_df, on='product_id', how='outer')
    df = df.merge(olist_customers_df, on='customer_id', how='outer')
    df = df.merge(olist_sellers_df, on='seller_id', how='outer')
    df = df.merge(olist_closed_deals_df, on='seller_id', how='left')
    df = df.merge(olist_marketing_qualified_leads_df, on='mql_id', how='left')
    return df

### Questions to explore

- Did a customer that purchased an item and received a delayed delivery purchase again?
- Are delayed deliveries purchased less?
- How do I connect a review of a random customer with new customers?
- Research on how does causal inference work on a "future" timeline.
- What are possible confounders?
- What are possible confounders that don’t exist in our data?

In [ ]:
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])

df['late_delivery_in_days'] = (df['order_delivered_customer_date'] \
                                            - df['order_estimated_delivery_date']).dt.days

# if late_delivery_in_days > 0, then delivery was late
df['is_delivery_late'] = np.where(df['late_delivery_in_days'] > 0, 1, 0)

In [5]:
df.shape

(119143, 55)